# Georgia Tech Mellea Tutorial

This tutorial will introduce the basics of Generative Computing through a series of labs. Everything you need is in this Notebook.

During this tutorial, we will:
1. Get up an running with Mellea.
2. See the Instruct - Validate - Repair pattern in action.
3. Encapsulate LLM calls using a functional interface via Mellea's `@generative` decorator.
4. Combine Mellea and Docling to write a generative program that operates over documents.
5. Encapsulate LLM calls using an object-oriented interface via Mellea's `MObject` protocol.

## Getting Started

Run the first cell during our introduction. The first cell will:
 * download an install ollama on your Colab instance
 * download the `granite3.3:8b` model weights


In [1]:
# Install ollama.
!curl -fsSL https://ollama.com/install.sh | sh > /dev/null
!nohup ollama serve >/dev/null 2>&1 &

# Download the granite:3.3:8b weights.
!ollama pull granite3.3:8b
!ollama pull llama3.2:3b

# install Mellea.
!uv pip install mellea[all] -q

# Run docling once to download model weights.
from mellea.stdlib.docs.richdocument import RichDocument

RichDocument.from_document_file("https://mellea.ai")

# Some UI niceness.
from IPython.display import HTML, display


def set_css():
    display(HTML("\n<style>\n pre{\n white-space: pre-wrap;\n}\n</style>\n"))


get_ipython().events.register("pre_run_cell", set_css)

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.




# LAB 1: Hello, Mellea!

Running `mellea.start_session()` initialize a new `MelleaSession`. The session holds three things:
1. The model to use for this session. In this tutorial we will use granite3.3:8b.
2. An inference engine; i.e., the code that actually calls our model. We will be using ollama, but you can also use Huggingface or any OpenAI-compatible endpoint.
3. A `Context`, which tells Mellea how to remember context between requests. This is sometimes called the "Message History" in other frameworks. Throughout this tutorial, we will be using a `SimpleContext`. In `SimpleContext`s, **every request starts with a fresh context**. There is no preserved chat history between requests. Mellea provides other types of context, but today we will not be using those features. See the Tutorial for further details.

In [2]:
import mellea

m = mellea.start_session()

answer = m.chat(
    "Tell me some fun historical trivia about the history of Computer Science at Georgia Tech."
)
print(answer.content)

1. Pioneering Research: Georgia Institute of Technology (Georgia Tech) has a rich history in computer science, dating back to its early contributions. In the 1950s, the school was among the first to offer an undergraduate degree in electrical engineering with a concentration in computer science. This program laid the groundwork for modern computer science education.

2. The Johns-Manville Research Laboratory: In the 1960s, Georgia Tech established the Johns-Manville Research Laboratory, a joint venture with Johns Manville, an asbestos manufacturing company. This lab focused on artificial intelligence (AI) and expert systems research, making Georgia Tech one of the early institutions to delve into AI.

3. The Center for Reliable and High-Performance Computing (CRHPC): Founded in 1985, CRHPC was one of the first centers dedicated to high-performance computing at a university level. It played a significant role in advancing parallel processing and distributed systems research.

4. The Gra

# LAB 2: Instruct-Validate-Repair

Instruct-Validate-Repair is a design pattern for building robust automation using LLMs. The idea is simple:
1. Instruct the model to perform a task and specify requirements on the output of the task.
2. Validate that these requirements are satisfied by the model's output.
3. If any requirements fail, try to repair.

In [3]:
import mellea
from mellea.stdlib.requirement import check, req, simple_validate
from mellea.stdlib.sampling import RejectionSamplingStrategy

requirements = [
    req("The email should have a salutation"),  # == r1
    req(
        "Use only lower-case letters",
        validation_fn=simple_validate(lambda x: x.lower() == x),
    ),  # == r2
    check("Do not mention purple elephants."),  # == r3
]


def write_email(m: mellea.MelleaSession, name: str, notes: str) -> str:
    email_candidate = m.instruct(
        "Write an email to {{name}} using the notes following: {{notes}}.",
        requirements=requirements,
        strategy=RejectionSamplingStrategy(loop_budget=5),
        user_variables={"name": name, "notes": notes},
        return_sampling_results=True,
    )
    if email_candidate.success:
        return str(email_candidate.result)
    else:
        return email_candidate.sample_generations[0].value


m = mellea.start_session()
print(
    write_email(
        m,
        "Olivia",
        "Olivia helped the lab over the last few weeks by organizing intern events, advertising the speaker series, and handling issues with snack delivery.",
    )
)

  0%|          | 0/5 [00:00<?, ?it/s]

=== 19:48:50-INFO ======
FAILED. Valid: 2/3


INFO:fancy_logger:FAILED. Valid: 2/3
 20%|██        | 1/5 [00:10<00:40, 10.15s/it]

=== 19:48:56-INFO ======
FAILED. Valid: 2/3


INFO:fancy_logger:FAILED. Valid: 2/3
 40%|████      | 2/5 [00:16<00:24,  8.19s/it]

...

=== 19:49:24-INFO ======
FAILED. Valid: 2/3


INFO:fancy_logger:FAILED. Valid: 2/3
100%|██████████| 5/5 [00:44<00:00,  8.82s/it]

=== 19:49:24-INFO ======
Invoking select_from_failure after 5 failed attempts.



INFO:fancy_logger:Invoking select_from_failure after 5 failed attempts.


Subject: heartfelt thanks for your recent support

dear olivia,

i hope this message finds you well. i am writing to express our lab's deepest gratitude for the incredible assistance you've provided over the past few weeks. your dedication in organizing intern events has been instrumental in ensuring smooth operations and fostering a collaborative environment.

additionally, your efforts in advertising the speaker series have significantly boosted attendance and engagement, highlighting your knack for effective communication and promotion. not to mention, your handling of snack delivery issues showcased your problem-solving skills and commitment to detail, ensuring even the small things run seamlessly.

we are truly fortunate to have had your support, and we genuinely appreciate all that you've done. thank you once again for your hard work and positive contributions.

warm regards,

[Your Name]
lab coordinator


## LAB 3: Writing Compositional Code with Generative Stubs

In classical programming, pure (stateless) functions are a simple and powerful abstraction. A pure function takes inputs, computes outputs, and has no side effects. Generative programs can also use functions as abstraction boundaries, but in a generative program the meaning of the function can be given by an LLM instead of an interpreter or compiler. This is the idea behind a GenerativeSlot.

A GenerativeSlot is a function whose implementation is provided by an LLM. In Mellea, you define these using the `@generative` decorator. The function signature specifies the interface, and the docstring (or type annotations) guide the LLM in producing the output.

In this lab, we will see how **compositionality checks** can be used to combine libraries of generative functions.

In [4]:
from mellea import generative

################################################################################
# SUMMARIZER LIBRARY                                                           #
################################################################################


@generative
def summarize_meeting(transcript: str) -> str:
    """Summarize the meeting transcript into a concise paragraph of main points."""


@generative
def summarize_contract(contract_text: str) -> str:
    """Produce a natural language summary of contract obligations and risks."""


@generative
def summarize_short_story(story: str) -> str:
    """Summarize a short story, with one paragraph on plot and one paragraph on broad themes."""


################################################################################
# DECISION AIDES LIBRARY                                                       #
################################################################################


# The Decision Aides Library
@generative
def propose_business_decision(summary: str) -> str:
    """Given a structured summary with clear recommendations, propose a business decision."""


@generative
def generate_risk_mitigation(summary: str) -> str:
    """If the summary contains risk elements, propose mitigation strategies."""


@generative
def generate_novel_recommendations(summary: str) -> str:
    """Provide a list of novel recommendations that are similar in plot or theme to the short story summary."""

### Summarizing a meeting

Let's use the meeting summarizer to summarize a meeting transcript.

In [5]:
################################################################################
# Transcript of a meeting discussing risks                                     #
################################################################################

transcript = """Meeting Transcript: Market Risk Review -- Self-Sealing Stembolts Division
Date: December 1, 3125
Attendees:

Karen Rojas, VP of Product Strategy

Derek Madsen, Director of Global Procurement

Felicia Zheng, Head of Market Research

Tom Vega, CFO

Luis Tran, Engineering Liaison

Karen Rojas:
Thanks, everyone, for making time on short notice. As you've all seen, we've got three converging market risks we need to address: tariffs on micro-carburetors, increased adoption of the self-interlocking leafscrew, and, believe it or not, the "hipsterfication" of the construction industry. I need all on deck and let's not waste time. Derek, start.

Derek Madsen:
Right. As of Monday, the 25% tariff on micro-carburetors sourced from the Pan-Alpha Centauri confederacy is active. We tried to pre-purchase a three-month buffer, but after that, our unit cost rises by $1.72. That's a 9% increase in the BOM cost of our core model 440 stembolt. Unless we find alternative suppliers or pass on the cost, we're eating into our already narrow margin.

Tom Vega:
We cannot absorb that without consequences. If we pass the cost downstream, we risk losing key mid-tier OEM clients. And with the market already sniffing around leafscrew alternatives, this makes us more vulnerable.

Karen:
Lets pause there. Felicia, give us the quick-and-dirty on the leafscrew.

Felicia Zheng:
It's ugly. Sales of the self-interlocking leafscrew—particularly in modular and prefab construction—are up 38% year-over-year. It's not quite a full substitute for our self-sealing stembolts, but they are close enough in function that some contractors are making the switch. Their appeal? No micro-carburetors, lower unit complexity, and easier training for install crews. We estimate we've lost about 12% of our industrial segment to the switch in the last two quarters.

Karen:
Engineering, Luis; your take on how real that risk is?

Luis Tran:
Technically, leafscrews are not as robust under high-vibration loads. But here's the thing: most of the modular prefab sites don not need that level of tolerance. If the design spec calls for durability over 10 years, we win. But for projects looking to move fast and hit 5-year lifespans? The leafscrew wins on simplicity and cost.

Tom:
So they're eating into our low-end. That's our volume base.

Karen:
Exactly. Now let's talk about this last one: the “hipsterfication” of construction. Felicia?

Felicia:
So this is wild. We're seeing a cultural shift in boutique and residential construction—especially in markets like Beckley, West Sullivan, parts of Osborne County, where clients are requesting "authentic" manual fasteners. They want hand-sealed bolts, visible threads, even mismatched patinas. It's an aesthetic thing. Function is almost secondary. Our old manual-seal line from the 3180s? People are hunting them down on auction sites.

Tom:
Well, I'm glad I don't have to live in the big cities... nothing like this would ever happen in downt-to-earth places Brooklyn, Portland, or Austin.

Luis:
We literally got a request from a design-build firm in Keough asking if we had any bolts “pre-distressed.”

Karen:
Can we spin this?

Tom:
If we keep our vintage tooling and market it right, maybe. But that's niche. It won't offset losses in industrial and prefab.

Karen:
Not yet. But we may need to reframe it as a prestige line—low volume, high margin. Okay, action items. Derek, map alternative micro-carburetor sources. Felicia, get me a forecast on leafscrew erosion by sector. Luis, feasibility of reviving manual seal production. Tom, let's scenario-plan cost pass-through vs. feature-based differentiation.

Let's reconvene next week with hard numbers. Thanks, all."""

summary = summarize_meeting(m, transcript=transcript)
print(f"Summary of meeting: {summary}")

Summary of meeting: Action items: Derek to map alternative micro-carburetor sources; Felicia to provide forecast on leafscrew erosion by sector; Luis to assess feasibility of reviving manual seal production; Tom to scenario-plan cost pass-through vs. feature-based differentiation. Reconvene next week with hard numbers.


### Composing Summarizers with decision aides

In [6]:
################################################################################
# COMPOSITIONALITY CHECKS                                                      #
################################################################################
from typing import Literal


@generative
def has_structured_conclusion(summary: str) -> Literal["yes", "no"]:
    """Determine whether the summary contains a clearly marked conclusion or recommendation."""


@generative
def contains_actionable_risks(summary: str) -> Literal["yes", "no"]:
    """Check whether the summary contains references to business risks or exposure."""


@generative
def has_theme_and_plot(summary: str) -> Literal["yes", "no"]:
    """Check whether the summary contains both a plot and thematic elements."""


################################################################################
# APPLY DECISION AIDES                                                      #
################################################################################

# generate risk mitigation straegies based upon the meeting summary.
if contains_actionable_risks(m, summary=summary) == "yes":
    mitigation = generate_risk_mitigation(m, summary=summary)
    print(f"Mitigation: {mitigation}")
else:
    print("Summary does not contain actionable risks.")

Mitigation: Potential risks identified: alternative micro-carburetor sourcing, leafscrew erosion impact on sectors, feasibility of manual seal production revival, and cost pass-through vs feature-based differentiation strategies. Proposed mitigation strategies: Derek to evaluate multiple alternative suppliers for micro-carburetors; Felicia to analyze historical data and trends for leafscrew erosion predictions by sector; Luis to conduct a comprehensive cost-benefit analysis for manual seal production revival; Tom to model various scenarios for cost pass-through and feature-based differentiation strategies. Reconvene next week to review these analyses and make informed decisions.


In [7]:
if has_structured_conclusion(m, summary=summary) == "yes":
    decision = propose_business_decision(m, summary=summary)
    print(f"Decision: {decision}")
else:
    print("Summary lacks a structured conclusion.")

Decision: Based on the proposed action items, the business decision could be to evaluate alternative micro-carburetor sources, assess the feasibility of reviving manual seal production, and scenario plan cost pass-through versus feature-based differentiation strategies. The final decision should incorporate hard numbers obtained after reconvene next week.


# LAB 4: Docling and Mellea

In this lab, we will use both Docling and Mellea to extract and then modify data in a PDF.

In [8]:
from mellea.stdlib.docs.richdocument import RichDocument

rd = RichDocument.from_document_file("https://arxiv.org/pdf/1906.04043")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## Extract table from the document

We can use docling to extract documents from the table.

In [ ]:
from mellea.stdlib.docs.richdocument import Table

table1: Table = rd.get_tables()[0]
print(table1.to_markdown())

# Work with the Table Object

The Table object is Mellea-ready and can be used immediately with LLMs. In this example, table1 is transformed to have an extra column "Model" which contains the model string from the Feature column or "None" if there is none.

In [ ]:
from mellea.backends.model_ids import META_LLAMA_3_2_3B
from mellea.backends.ollama import OllamaModelBackend
from mellea.backends.types import ModelOption

# You can use multiple different models at the same time!
m_llama = mellea.MelleaSession(backend=OllamaModelBackend(model_id=META_LLAMA_3_2_3B))

for seed in [x * 12 for x in range(5)]:
    table2 = m_llama.transform(
        table1,
        "Add a 'Model' column as the last column that extracts which model was used for that feature or 'None' if none.",
        model_options={ModelOption.SEED: seed},
    )
    if isinstance(table2, Table):
        print(table2.to_markdown())
    else:
        print("==== TRYING AGAIN after non-useful output.====")

The model has fulfilled the task and coming back with a parsable syntax. You could now call (e.g. m.query(table2, "Are there any GPT models referenced?")) or continue transformation (e.g. m.transform(table2, "Transpose the table.")).


# LAB 5: Generative Objects

Object-oriented programming (OOP) is a powerful paradigm for organizing code: you group related data and the methods that operate on that data into classes. In the world of LLMs, a similar organizational principle emerges—especially when you want to combine structured data with LLM-powered "tools" or operations. This is where Mellea's MObject abstraction comes in.

**The MObject Pattern**: You should store data alongside its relevant operations (tools). This allows LLMs to interact with both the data and methods in a unified, structured manner. It also simplifies the process of exposing only the specific fields and methods you want the LLM to access.

The MObject pattern also provides a way of evolving existing classical codebases into generative programs. Mellea's @mify decorator lets you turn any class into an MObject. If needed, you can specify which fields and methods are included, and provide a template for how the object should be represented to the LLM.

In [ ]:
from io import StringIO

import pandas as pd

from mellea.stdlib.mify import mify


@mify(fields_include={"table"}, template="{{ table }}")
class MyCompanyDatabase:
    table: str = """| Store      | Sales   |
| ---------- | ------- |
| Northeast  | $250    |
| Southeast  | $80     |
| Midwest    | $420    |"""

    def __init__(self, *, table: str | None = None):
        if table is not None:
            self.table = table

    def _parse_table(self, table: str) -> pd.DataFrame:
        # Clean up the markdown table
        # Drop the separator row and strip whitespace/pipes
        lines = [
            line.strip()
            for line in table.strip().splitlines()
            if not set(line.strip()) <= {"|", "-", " "}
        ]
        cleaned = "\n".join(lines)

        # Read into dataframe
        df = pd.read_csv(StringIO(cleaned), sep="|")
        df = df.rename(columns=lambda x: x.strip())  # strip spaces from column names
        df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
        return df

    def update_sales(self, store: str, amount: str):
        """Update the sales for a specific store."""
        df = self._parse_table(self.table)
        df.loc[df["Store"] == store, "Sales"] = amount
        return MyCompanyDatabase(table=df.to_csv(sep="|", index=False, header=True))

In [ ]:
db = MyCompanyDatabase()
print(m.query(db, "What were sales for the Northeast branch this month?"))
db = m.transform(db, "Update the northeast sales to 1250.")
print(m.query(db, "What were sales for the Northeast branch this month?"))